In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# 단계 1: 폰트 설치
import matplotlib.font_manager as fm
import matplotlib.pyplot as plt
!apt-get -qq -y install fonts-nanum > /dev/null
#fontpath = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf'

#font = fm.FontProperties(fname=fontpath, size=9)

#fm._rebuild()



fe = fm.FontEntry(
    fname=r'/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf', # ttf 파일이 저장되어 있는 경로
    name='NanumGothic')                        # 이 폰트의 원하는 이름 설정
fm.fontManager.ttflist.insert(0, fe)              # Matplotlib에 폰트 추가
plt.rcParams.update({'font.size': 18, 'font.family': 'NanumGothic'}) # 폰트 설


In [ ]:
# 단계 2: 런타임 재시작
import os
os.kill(os.getpid(), 9)

In [ ]:
# 단계 3: 한글 폰트 설정
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.font_manager as fm

# 마이너스 표시 문제
mpl.rcParams['axes.unicode_minus'] = False

# 한글 폰트 설정
fe = fm.FontEntry(
    fname=r'/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf', # ttf 파일이 저장되어 있는 경로
    name='NanumGothic')                        # 이 폰트의 원하는 이름 설정
fm.fontManager.ttflist.insert(0, fe)              # Matplotlib에 폰트 추가
plt.rcParams.update({'font.size': 18, 'font.family': 'NanumGothic'}) # 폰트 설

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

df = pd.read_csv('/content/빅데이터이음.csv', encoding='utf-8')

print(df)


             일자         지역  문화취미-영화관      숙박       약국        연료  유통업영리 - 편의점  \
0    2025-01-01  인천광역시 강화군     61000  310100   501400   7369451      5430590   
1    2025-01-02  인천광역시 강화군     54000   62000  3396110  12740492      6514280   
2    2025-01-03  인천광역시 강화군      7000  240000  3793110  10325810      6818890   
3    2025-01-04  인천광역시 강화군     86000  250100  2719490  11967997      6222940   
4    2025-01-05  인천광역시 강화군      7000   20000   434900   5800303      4768610   
..          ...        ...       ...     ...      ...       ...          ...   
336  2025-01-27         기타         0       0        0         0        36150   
337  2025-01-28         기타         0       0        0         0       104950   
338  2025-01-29         기타         0       0        0         0       149650   
339  2025-01-30         기타         0       0        0         0        71200   
340  2025-01-31         기타         0       0        0         0        81910   

         일반휴게        학원        기타      

In [ ]:
df.columns

Index(['일자', '지역', '문화취미-영화관', '숙박', '약국', '연료', '유통업영리 - 편의점', '일반휴게', '학원',
       '기타', '전체금액'],
      dtype='object')

In [ ]:
df.head(1)

,일자,지역,문화취미-영화관,숙박,약국,연료,유통업영리 - 편의점,일반휴게,학원,기타,전체금액
0,2025-01-01,인천광역시 강화군,61000,310100,501400,7369451,5430590,36367190,398000,38417950,88855681


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import numpy as np

# Matplotlib 한글 깨짐 방지 설정 및 스타일 적용
plt.rcParams['font.family'] = 'NanumGothic'
plt.rcParams['axes.unicode_minus'] = False
plt.style.use('ggplot')

# 1. 세 파일 로드, 결합 및 전처리

def load_and_combine_data_final():

    df_list = []

    #2024-11 데이터 로드 (bigdata.csv
    try:
        try:
            df_nov = pd.read_csv("bigdata.csv", encoding='utf-8')
        except UnicodeDecodeError:
            df_nov = pd.read_csv("bigdata.csv", encoding='EUC-KR')

        df_list.append(df_nov)
        print("2024-11 데이터 (bigdata.csv) 로드 성공.")
    except Exception as e:
        print(f"2024-11 데이터 로드 실패: {e}")

    #2024-12 데이터 로드 (빅데이터이음2.csv)
    try:
        df_dec = pd.read_csv("빅데이터이음2.csv", encoding='utf-8')
        df_list.append(df_dec)
        print("2024-12 데이터 (빅데이터이음2.csv) 로드 성공 (UTF-8).")
    except Exception as e:
        print(f"2024-12 데이터 로드 실패: {e}")

    # 2025-01 데이터 로드 (빅데이터이음.csv)
    try:
        df_jan = pd.read_csv("빅데이터이음.csv", encoding='utf-8')
        df_list.append(df_jan)
        print("2025-01 데이터 (빅데이터이음.csv) 로드 성공 (UTF-8).")
    except Exception as e:
        print(f" 2025-01 데이터 로드 실패: {e}")

    if len(df_list) < 3:
        raise FileNotFoundError(f"파일 중 {3 - len(df_list)}개 로드 실패 ")

    # 세 DataFrame 결합
    df_combined = pd.concat(df_list, ignore_index=True)

    # 결합된 데이터 전처리
    df_combined['일자'] = df_combined['일자'].astype(str).str.replace('.', '-', regex=False)
    df_combined['일자'] = pd.to_datetime(df_combined['일자'])

    df_combined['월'] = df_combined['일자'].dt.to_period('M').astype(str)
    df_combined['군구'] = df_combined['지역'].apply(lambda x: x.split()[-1])

    return df_combined


# 2. 월별 분석 및 요약 데이터 생성 (

print("\n--- 2. 분석 및 요약 데이터 생성 시작 ---")
industry_cols = ['문화취미-영화관', '숙박', '약국', '연료', '유통업영리 - 편의점', '일반휴게', '학원', '기타']
group_cols = ['전체금액'] + industry_cols
months_to_analyze = ['2024-11', '2024-12', '2025-01']
monthly_data = {}

# 월별로 데이터프레임을 나누고 분석
for month in months_to_analyze:
    df_month = df_data[df_data['월'] == month]

    # 2-1. 월별 업종 요약
    df_ind = df_month[industry_cols].sum().reset_index()
    df_ind.columns = ['업종', '총 소비 금액']
    df_ind = df_ind.sort_values(by='총 소비 금액', ascending=False)
    df_ind['총 소비 금액 (억 원)'] = (df_ind['총 소비 금액'] / 100000000).round(2)

    # 2-2. 월별 군구 요약
    df_dist = df_month.groupby('군구')[group_cols].sum().reset_index()
    df_dist = df_dist.sort_values(by='전체금액', ascending=False)
    df_dist['전체금액 (억 원)'] = (df_dist['전체금액'] / 100000000).round(2)

    monthly_data[month] = {'industry': df_ind, 'district': df_dist}

# 2-3. 전체 월별 총 소비 금액 요약
df_monthly_summary = df_data.groupby('월')[group_cols].sum().reset_index()
df_monthly_summary['전체금액 (억 원)'] = (df_monthly_summary['전체금액'] / 100000000).round(2)
df_monthly_summary.to_csv("월별 총 소비 금액.csv", index=False, encoding='utf-8-sig')


# 3. 데이터 시각화

print("\n--- 3. 데이터 시각화 시작 (8개 PNG 파일 생성) ---")


# 3-1. 월별 업종별 총 소비 금액 시각화
for month in months_to_analyze:
    df_ind = monthly_data[month]['industry']
    file_name = f'result1{month}.png'

    plt.figure(figsize=(12, 6))
    sns.barplot(
        x='업종',
        y='총 소비 금액 (억 원)',
        data=df_ind,
        palette='viridis'
    )
    plt.title(f'업종별 총 소비 금액 ({month})', fontsize=16)
    plt.xlabel('업종', fontsize=12)
    plt.ylabel('총 소비 금액 (억 원)', fontsize=12)
    plt.xticks(rotation=45, ha='right')
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.tight_layout()
    plt.savefig(file_name)
    plt.close()


# 3-2. 월별 전체 소비 금액 비교 (선 그래프)
plt.figure(figsize=(8, 6))
df_monthly_summary['월_sort'] = pd.to_datetime(df_monthly_summary['월'])
df_monthly_summary = df_monthly_summary.sort_values('월_sort')

sns.lineplot(
    x='월',
    y='전체금액 (억 원)',
    data=df_monthly_summary,
    marker='o',
    color='blue',
    linewidth=3
)
plt.title('월별 총 소비 금액 추이 (2024.11 vs 2024.12 vs 2025.01)', fontsize=16)
plt.xlabel('월', fontsize=12)
plt.ylabel('총 소비 금액 (억 원)', fontsize=12)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.savefig('result2.png')
plt.close()



# 3-3. 일별-월별 소비 금액 비교 시각화
df_data['일'] = df_data['일자'].dt.day
df_daily_monthly_total = df_data.groupby(['월', '일'])['전체금액'].sum().reset_index()
df_daily_monthly_total['일별 총 소비 금액 (억 원)'] = df_daily_monthly_total['전체금액'] / 100000000

plt.figure(figsize=(16, 8))
sns.lineplot(
    x='일',
    y='일별 총 소비 금액 (억 원)',
    hue='월', # 월별로 선을 구분
    data=df_daily_monthly_total,
    marker='o',
    linewidth=2.5,
    palette='Set1'
)
plt.title('일자별 총 소비 금액 추이 (2024년 11월, 12월, 2025년 1월 비교)', fontsize=18)
plt.xlabel('일 (Day of Month)', fontsize=14)
plt.ylabel('일별 총 소비 금액 (억 원)', fontsize=14)
plt.xticks(np.arange(1, 32, 1))
plt.legend(title='월 (Month)', loc='upper left')
plt.grid(axis='both', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.savefig('result3.png')
plt.close()



# 3-4. 월별 군/구별 업종별 소비 금액 비중 시각화 (Stacked Bar Plot 3개)
for month in months_to_analyze:
    df_dist = monthly_data[month]['district']
    file_name = f'result4{month}.png'

    # '군구'를 인덱스로 설정하고 업종별 금액만 선택
    df_plot = df_dist.set_index('군구')[industry_cols]

    # 금액 단위를 '억 원'으로 변환하여 시각화 가독성 높이기
    df_plot_billion = df_plot / 100000000

    # Pandas의 plot 메서드를 사용하여 Stacked Bar Plot 생성
    plt.figure(figsize=(15, 8))
    ax = df_plot_billion.plot(
        kind='bar',
        stacked=True,
        figsize=(15, 8),
        colormap='Spectral',
        edgecolor='black',
        ax=plt.gca()
    )

    # 제목 및 레이블 설정
    plt.title(f'군/구별 업종별 소비 금액 비중 ({month})', fontsize=16)
    plt.xlabel('군/구', fontsize=12)
    plt.ylabel('총 소비 금액 (억 원)', fontsize=12)
    plt.xticks(rotation=45, ha='right')

    # 범례 위치를 그래프 밖으로 이동하여 겹치지 않게 설정
    plt.legend(title='업종', bbox_to_anchor=(1.05, 1), loc='upper left')

    # 그래프 저장
    plt.tight_layout(rect=[0, 0, 0.85, 1])
    plt.savefig(file_name)
    plt.close()





--- 2. 분석 및 요약 데이터 생성 시작 ---

--- 3. 데이터 시각화 시작 (8개 PNG 파일 생성) ---


/tmp/ipython-input-1885718387.py:105: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.barplot(
/tmp/ipython-input-1885718387.py:105: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.barplot(
/tmp/ipython-input-1885718387.py:105: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.barplot(
